In [197]:
import pandas as pd

In [198]:
#importing dataset from https://www.kaggle.com/datasets/nehaprabhavalkar/indian-food-101
data = pd.read_csv("/content/indian_food.csv")
data.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [199]:
#dropping unwanted columns
data = data.drop(columns=['flavor_profile','region','state', 'diet'])
data

,name,ingredients,prep_time,cook_time,course
0,Balu shahi,"Maida flour, yogurt, oil, sugar",45,25,dessert
1,Boondi,"Gram flour, ghee, sugar",80,30,dessert
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",15,60,dessert
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",15,30,dessert
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",15,40,dessert
...,...,...,...,...,...
250,Til Pitha,"Glutinous rice, black sesame seeds, gur",5,30,dessert
251,Bebinca,"Coconut milk, egg yolks, clarified butter, all...",20,60,dessert
252,Shufta,"Cottage cheese, dry dates, dried rose petals, ...",-1,-1,dessert
253,Mawa Bati,"Milk powder, dry fruits, arrowroot powder, all...",20,45,dessert


In [200]:
#data exploration & cleaning
print(data.course.unique())
print(data.loc[data['course']=='snack'])

['dessert' 'main course' 'starter' 'snack']
                  name                                        ingredients  \
88                Poha  Beaten rice flakes, potato, curry leaves, gree...   
89                Fara  Chana dal, whole wheat flour, arhar dal, white...   
90             Kachori  Moong dal, rava, garam masala, dough, fennel s...   
109          Pani puri      Kala chana, mashed potato, boondi, sev, lemon   
111              Papad       Urad dal, sev, lemon juice, chopped tomatoes   
117             Samosa  Potatoes, green peas, garam masala, ginger, dough   
124               Attu  Whole wheat flour, arhar dal, ginger, kala jee...   
128               Dosa  Chana dal, urad dal, whole urad dal, blend ric...   
129           Idiappam              Rice flour, hot water, grated coconut   
130               Idli  Split urad dal, urad dal, idli rice, thick poh...   
131              Kanji     Carrot, yellow mustard, red chilli, black salt   
144        Masala Dosa  Chana da

In [201]:
#finding data types
print(data.dtypes)

name           object
ingredients    object
prep_time       int64
cook_time       int64
course         object
dtype: object


In [202]:
#removing spaces from string and lowering the case
str_cols = list(data.columns)
str_cols.remove('prep_time')
str_cols.remove('cook_time')
for i in str_cols:
    data[i] = data[i].str.strip()
    data[i] = data[i].str.lower()

In [203]:
#removing missing or correcting garbage value
#data['prep_time'].value_counts()[-1]
#data = data[data.cook_time != -1]
for i in ['name']:
  data[i] = data[i].str.replace(" ", "-")
for i in ['course']:
  data[i] = data[i].str.replace(" ", "-")
for i in ['ingredients']:
  data[i] = data[i].str.replace(" ", "-")
  data[i] = data[i].str.replace(",-", "_")
  data[i] = data[i].str.replace("_", ", ")
data['ingredients'] = list(data['ingredients'])

In [204]:
data

,name,ingredients,prep_time,cook_time,course
0,balu-shahi,"maida-flour, yogurt, oil, sugar",45,25,dessert
1,boondi,"gram-flour, ghee, sugar",80,30,dessert
2,gajar-ka-halwa,"carrots, milk, sugar, ghee, cashews, raisins",15,60,dessert
3,ghevar,"flour, ghee, kewra, milk, clarified-butter, su...",15,30,dessert
4,gulab-jamun,"milk-powder, plain-flour, baking-powder, ghee,...",15,40,dessert
...,...,...,...,...,...
250,til-pitha,"glutinous-rice, black-sesame-seeds, gur",5,30,dessert
251,bebinca,"coconut-milk, egg-yolks, clarified-butter, all...",20,60,dessert
252,shufta,"cottage-cheese, dry-dates, dried-rose-petals, ...",-1,-1,dessert
253,mawa-bati,"milk-powder, dry-fruits, arrowroot-powder, all...",20,45,dessert


In [205]:
from numpy import append
def easyA(x):
  available = []
  suggested = []
  recommendation = pd.DataFrame(columns = ['recipe', 'ingredients', 'score'])
  for i in range(0, len(data)):
    count = 0
    for j in x:
      if j in data['ingredients'][i]:
        count +=1
        available.append(j)
    if count >= 3:
      series = pd.DataFrame({'recipe': [data['name'][i]], 'ingredients': data['ingredients'][i], 'score': "{:.2f}".format((len(available))/(len(data['ingredients'][i]))*100)})
      recommendation = recommendation.append(series)
    available.clear()
  return recommendation

x = ["fenugreek", "oil", "salt", "yogurt", "sugar", "rice", "potatoes", "milk", "maida-flour"]
easyA(x)

,recipe,ingredients,score
0,balu-shahi,"maida-flour, yogurt, oil, sugar",12.90
0,kheer,"milk, rice, sugar, dried-fruits",9.68
0,lassi,"yogurt, milk, nuts, sugar",12.00
0,gavvalu,"rice-flour, sugar, salt, ghee, semolina",7.69
0,doodhpak,"milk, rice, sugar, dry-fruits",10.34
0,chak-hao-kheer,"rice, milk, sugar, cardamom",11.11
0,aloo-methi,"potato, fenugreek-leaves, chillies, salt, oil",6.67
0,payasam,"rice, cashew-nuts, milk, raisins, sugar",7.69
